<a href="https://colab.research.google.com/github/Leonild/DS_mAIstros/blob/main/Bayesian_Inference_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing data
Importing from cloud

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal as mvn

data_train = pd.read_csv('/content/drive/MyDrive/EnhanceIT/Data_training/MNIST_train.csv')

data_test = pd.read_csv('/content/drive/MyDrive/EnhanceIT/Data_training/MNIST_test.csv')

print(data_train.shape)
print(data_test.shape)

data_test=data_test.drop_duplicates()
data_train=data_train.drop_duplicates()

print(data_train.shape)
print(data_test.shape)

print(data_train.head())


(60000, 787)
(10000, 787)
(60000, 787)
(10000, 787)
   Unnamed: 0  index  labels  0  1  2  3  4  5  6  ...  774  775  776  777  \
0           0      0       5  0  0  0  0  0  0  0  ...    0    0    0    0   
1           1      1       0  0  0  0  0  0  0  0  ...    0    0    0    0   
2           2      2       4  0  0  0  0  0  0  0  ...    0    0    0    0   
3           3      3       1  0  0  0  0  0  0  0  ...    0    0    0    0   
4           4      4       9  0  0  0  0  0  0  0  ...    0    0    0    0   

   778  779  780  781  782  783  
0    0    0    0    0    0    0  
1    0    0    0    0    0    0  
2    0    0    0    0    0    0  
3    0    0    0    0    0    0  
4    0    0    0    0    0    0  

[5 rows x 787 columns]


# Baysian Solution

In [ ]:
class GaussBayes():

  def fit(self, X, y, epsilon = 1e-3):
    self.likelihoods = dict()
    self.priors = dict()

    self.K = set(y.astype(int))

    for k in self.K:
      X_k = X[y==k,:]
      N_k, D = X_k.shape
      mu_k = X_k.mean(axis=0)
      #calculating by covariance matrix
      self.likelihoods[k] = {"mean": X_k.mean(axis=0), 
                             "covariance": (1/(N_k - 1)) * np.matmul((X_k-mu_k).T,(X_k-mu_k))+
                             epsilon*np.identity(D)}
      self.priors[k] = len(X_k)/len(X)
      
  def predict(self, X):
    N, D = X.shape
    #probability verctor/matrix
    P_hat = np.zeros((N, len(self.K)))

    for k, l in self.likelihoods.items():
      P_hat[:,k] = mvn.logpdf(X, l["mean"], l["covariance"]) + np.log(self.priors[k])

    return P_hat.argmax(axis=1) #returning the hights probability

## Accuracy

In [ ]:
#What is the average time that they match
def accuracy(y, y_hat):
  return np.mean(y==y_hat)

## Testing solution

In [ ]:
X_train = data_train.to_numpy()

X_test = data_test.to_numpy()
#print(X_test)
print(set(X_train[:,2]))

y = X_train[:,2] #getting the labels
X_train = X_train[:,3:] # 3 cause the first 2 are indexes and the 3sd is the lable

y_test = X_test[:,2] #getting the labels
X_test = X_test[:,3:] # 3 cause the first 2 are indexes and the 3sd is the lable

#print(X_test)

MNIST_gnb = GaussBayes()

MNIST_gnb.fit(X_train,y)
y_hat = MNIST_gnb.predict(X_test)


accuracy(y_test, y_hat)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


0.78565

#KNN Solution

In [ ]:
class KNNClassifier():

  def fit(self, X, y):
    self.X = X
    self.y = y

  def predict(self, X, k, epsilon = 1e-3):
    N = len(X)
    y_hat = np.zeros(N)

    for i in range(N):
      # getting list of distances
      dist2 = np.sum((self.X - X[i]) ** 2, axis=1)
      idxt = np.argsort(dist2)[:k] # the k nearest
      # a list of the 1/distances
      gamma_k = 1 / (np.sqrt(dist2[idxt]+epsilon))
      y_hat[i] = np.bincount(self.y[idxt], weights=gamma_k).argmax()
    
    return y_hat


## Accuracy

In [ ]:
#What is the average time that they match
def accuracy(y, y_hat):
  return np.mean(y==y_hat)

## Testing Solution

In [ ]:
X_train = data_train.to_numpy()

X_test = data_test.to_numpy()
#print(X_test)
print(set(X_train[:,2]))

y = X_train[:,2] #getting the labels
X_train = X_train[:,3:] # 3 cause the first 2 are indexes and the 3sd is the lable

y_test = X_test[:,2] #getting the labels
X_test = X_test[:,3:] # 3 cause the first 2 are indexes and the 3sd is the lable

#print(X_test)

#MNIST_knn = KNNClassifier()

#MNIST_knn.fit(X_train,y)
#y_hat = MNIST_knn.predict(X_test,5)

#accuracy(y_test, y_hat) # 0.96 28m 28 s

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


## Tunning

In [ ]:
import time

# get the start time
st_total = time.time()
MNIST_knn = KNNClassifier()
MNIST_knn.fit(X_train,y)
for i in range(5,20,5):
  st = time.time() # get the start time
  print("Execution for k = ", i)
  y_hat = MNIST_knn.predict(X_test,i)
  print("Accuracy: ", accuracy(y_test, y_hat))
  et = time.time() # get the end time
  elapsed_time = et - st
  print('Execution time:', elapsed_time, 'seconds\n\n')



# get the end time
et_total = time.time()

# get the execution time
elapsed_time = et_total - st_total
print('Experiment time:', elapsed_time, 'seconds')


Execution for k =  5
Accuracy:  0.9691
Execution time: 3970.9969503879547 seconds


Execution for k =  10


KeyboardInterrupt: ignored